**Вопросы для повторения:**
* что такое `std::mutex`? зачем он нужен? что внутри?
* что такое `std::lock_guard`? чем не устраивают `std::mutex::lock`, `std::mutex::unlock`?
* предположим, у нас есть многопоточная очередь задач `MTTasksQueue`. В чём здесь проблема? Как её будем исправлять?

```c++
void run_queued_task(MTTasksQueue& q)
{
    if (!q.empty())
        run_task(q.pop());
}
```

* а в чём может быть проблема с той же самой очередью тут? Какие есть варианты её исправить?

```c++
class MTTasksQueue
{
public:
    using Task = std::function<void(void)>;
    
    ...

    void pop_and_run()
    {
        std::lock_guard guard(mtx);

        if (!tasks_queue.empty())
        {        
            const auto task = std::move(tasks_queue.back());
            tasks_queue.pop_back();
            task();
        }        
    }

private:
    std::mutex mtx;
    std::queue<Task> tasks_queue;
};
```

<details>
<summary>ответ</summary>
<p>recursive_mutex и реорганизация кода. Что такое и как устроен recursive_mutex? Когда его следует использовать? Почему в данном случае recursive_mutex - плохое решение? Если остаться на обычном mutex, как следует с ним поступить?</p>
</details>

* Что такое deadlock? Каким минимальным числом потоков и mutex-ов устроить deadlock?

<details>
<summary>замечание</summary>
<p>"на самом деле", дважды вызов lock у std::mutex на одном потоке - это не обязательно deadlock. Документация утверждает, что это UB, и _скорее всего_ вы получите deadlock, но некоторые реализации могут задетектить ситуацию и бросить exception, но в общем случае всё совсем плохо - это UB</p>
<p><a href="https://en.cppreference.com/w/cpp/thread/mutex/lock">proof</a></p>
</details>


* что такое и зачем нужны `shared_mutex`, `unique_lock`, `shared_lock` ?

<br />

### Многопоточность. Продвинутый материал.

Документация:
* https://en.cppreference.com/w/cpp/atomic
* https://en.cppreference.com/w/cpp/atomic/memory_order

Серия статей от Jeff Pershing на понимание atomics, memory model && lock free:
* https://preshing.com/20120515/memory-reordering-caught-in-the-act/
* https://preshing.com/20120612/an-introduction-to-lock-free-programming
* https://preshing.com/20120625/memory-ordering-at-compile-time/
* https://preshing.com/20120710/memory-barriers-are-like-source-control-operations/
* https://preshing.com/20120913/acquire-and-release-semantics/
* https://preshing.com/20120930/weak-vs-strong-memory-models/
* https://preshing.com/20121019/this-is-why-they-call-it-a-weakly-ordered-cpu/
* https://preshing.com/20130618/atomic-vs-non-atomic-operations/
* https://preshing.com/20130702/the-happens-before-relation/
* https://preshing.com/20130922/acquire-and-release-fences/
* https://preshing.com/20130823/the-synchronizes-with-relation/
* https://preshing.com/20130930/double-checked-locking-is-fixed-in-cpp11/
* https://preshing.com/20131125/acquire-and-release-fences-dont-work-the-way-youd-expect/
* https://preshing.com/20140709/the-purpose-of-memory_order_consume-in-cpp11/
* В конце своих статей Джефф даёт ссылки на полезные материалы, их тоже рекомендуется почитать. Не стоит рассчитывать, что управитесь со всем багажом знаний за полчасика.
* И в комментарии к статьям приходят специалисты (Herb Sutter, Tarvis Downs), и объясняют, в чём Джефф был не прав, поэтому комментарии желательно тоже смотреть.

Другие статьи:
* [Memory Barriers: a Hardware View for Software Hackers](http://www.rdrop.com/users/paulmck/scalability/paper/whymb.2010.07.23a.pdf)
* [LINUX KERNEL MEMORY BARRIERS](https://www.kernel.org/doc/Documentation/memory-barriers.txt)
* [Максим Хижинский. Lock-free Data Structures. Basics: Atomicity and Atomic Primitives](https://kukuruku.co/post/lock-free-data-structures-basics-atomicity-and-atomic-primitives/)
* [Bjarne Stroustrup про ABA-проблему и как её решать](http://www.stroustrup.com/isorc2010.pdf)

Доклады про многопоточность:
* [CppCon 2017: Fedor Pikus “C++ atomics, from basic to advanced. What do they really do?”](https://www.youtube.com/watch?v=ZQFzMfHIxng)
* [CppCon 2018: Bryce Adelstein Lelbach “The C++ Execution Model”](https://www.youtube.com/watch?v=FJIn1YhPJJc)
* [CppCon 2019: Bryce Adelstein Lelbach “The C++20 Synchronization Library”](https://www.youtube.com/watch?v=Zcqwb3CWqs4)

<br />

#### thread-based vs task-based модели организации вычислений

В первой лекции про многопоточность мы неявно затронули это разделение.

Повторим материал, вспомним задачу параллельного поиска максимума в массиве.

Для простоты демонстрации вынесем функцию, которая считает максимум на части массива:
    
```c++
int max_by_block(const std::vector<int>& arr, const unsigned block_ix, const unsigned blocks_count)
{
    const unsigned block_size = arr.size() / blocks_count;
    
    // для простоты демонстрации, чтобы не обрабатывать хвосты и ошибки
    assert(block_size);
    assert(block_size * blocks_count == arr.size());
    assert(block_ix < blocks_count);

    const unsigned start_ix = block_size * block_ix;
    return *max_element(begin(arr) + start_ix,
                        begin(arr) + start_ix + block_size);
}
```

Решение через потоки:

```c++
int parallel_max(const std::vector<int>& arr, const unsigned threads_count)
{
    // maximum per threads
    std::vector<int> results(threads_count, 0);

    // create threads to search for maximum
    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([i, threads_count, &results](){
            results[i] = max_by_block(arr, i, threads_count);
        });

    // wait for threads to complete
    for (auto& t : threads)
        t.join();

    return *max_element(begin(results), end(results));
}
```

Решение через async:

```c++
int parallel_max(const std::vector<int>& arr, const unsigned tasks_count)
{
    // создать |tasks_count| фоновых задач
    std::vector<std::future<int>> futures;
    for (unsigned i = 0; i < tasks_count; ++i)
        futures.emplace_back(std::async(std::launch::async,
                                        [&arr, =]() { return max_by_block(arr, i, tasks_count); }));

    // collect results and reduce
    int res = INT_MIN;
    for (auto& f: futures)
        res = std::max(res, f.get());

    return res;
}
```

Вы можете отметить, что визуально разница не так уж и велика, но нужно отметить следующее:
    
* в решении через потоки:
    * строго зафиксировано число потоков, на котором запускаются вычисления
    * строго задан момент создания потоков и момент их уничтожения
    * пользователь выражает свои намерения в терминах _"создать поток, запустить вычисления, уничтожить поток"_
    

* в решении через `async` (через задачи) от пользователя скрывается информация 
    * от пользователя через интерфейс async спрятана (почти) информация сколько создаётся потоков, когда они создаются, продолжают ли они жить после выполнения задачи
    * пользователь выражает свои намерения в терминах _"выполнить фоновую задачу"_

<br />

##### executors

Развитие task-based моделей организации служат Executors - классы / мезанизмы управления фоновыми задачами.

Пока что в С++ executors не стандартизированы, только в процессе, поэтому каждый пишет во что горазд. В целом executor может позволять делать следующее:
* запуск фоновой задачи
* отмена фоновой задачи
* поддержка приоритетов задач
* поддержка зависимостей между задачами
* управление внутренним пулом потоков:
    * фиксированное число потоков / плавающее число потоков
    * момент создания и уничтожения потоков
* управление внутренней очередью задач

Ориентировочный вариант интерфейса executor (понятно, тут у каждого свои вариации):
    
```c++
class Executor
{
public:
    // кол-во потоков для выполнения задач задаём в конструкторе,
    // пусть он сам принимает решение, когда система перегружена,
    // и требуется отсыпать ещё потоков, либо наоборот, задачи
    // кончились и потоки можно уничтожить, высвободить память
    Executor(const unsigned min_threads_count,
             const unsigned max_threads_count);
    
    // запланировать выполнение задачи
    // (вариант, принятый +- в chromium)
    //
    // Callable - может быть std::function или
    // std::packaged_task или любой другой класс,
    // умеющий представлять собой некоторую задачу
    //
    // TaskParams - набор параметров задачи, могут,
    // например, содержать приоритет, отладочную
    // инфомрацию и другие параметры, отвечающие на
    // вопрос "как выполнить эту задачу"
    //
    // Callback - куда отсылать результат задачи
    template<typename Callable, typename Callback>
    bool post_task(Callable&& task, TaskParams params, Callback&& callback);
    
    // вариант реализации через std::future
    // (+- более в стиле std::async)
    template<typename Result, typename Callable>
    std::future<Result> post_task(Callable&& task, TaskParams params);
};
```

Можно, как вариант, создать экземпляр класса `Executor` как глобальную переменную. И надо аккуратно проработать потокобезопасность такого решения (либо однопоточный доступ до объекта либо потокобезопасная реализация)

<br />

Когда пользоваться подходом через потоки, когда - через задачи - зависит от... задачи.


**Пример**, когда решение через потоки может быть лучше:

Параллельный поиск оптимума в горячем месте программы
  * нужно строго контроллировать количество потоков
  * нужно строго контроллировать время их жизни
  * нужна синхронизация меджу потоками


**Пример**, когда решение через задачи может быть лучше:

Мы пишем браузер.

* <i>задача 1</i>: пользователь открыл сайт, нужно распарсить ответ от сервера, затем распарсить html-ку и отдать её на рендеринг.

  Нельзя парсить в главном потоке программы, иначе UI подвиснет и не будут реагировать кнопки меню. Нужно парсить в фоне.
  
  Скорость выполнения этой задачи важна для пользователя, её влияние на UX приложения критично.
  
  <i>Решение</i>: запостить фоновую задачу парсинга ответа и html-ки с высоким приоритетом.
  
  <i>Псевдокод:</i>
  
```c++
void process_server_response(const Response& response)
{
    global_executor.post_task(bind(parse_server_response, response),
                              TaskParams{ HIGH_PRIORITY },
                              process_raw_html);
}

void process_raw_html(const std::string& html)
{
    global_executor.post_task(bind(parse_html, html),
                              TaskParams{ HIGH_PROPRITY },
                              render_html);
}

void render_html(const HTML& html)
{
    // rendering code ...
}
```


* <i>задача 2</i>: пользователь запускает установку расширения в браузере.

  Установка сторонних расширений - процесс длительный. Не критично, если расширение не установится моментально, но и час установку ждать никто не будет.

  <i>Решение</i>: запостить фоновую задачу по установке расширения со средним приоритетом.
  
  
* <i>задача 3</i>: Браузер периодически запускает очистку данных на диске (протухшие кеши страниц / изображений, удалённые оставшиеся на диске расширения, прочистка БД от мусора)

  Цель - освободить место на диске, которое занято мусором, и слегка ускорить работу, уменьшив объём данных.
  
  Пользователю всё равно сколько будет идти задача, он даже не знает о её существовании.
  
  <i>Решение</i>: запостить фоновую задачу очистки с самым низким приоритетом. Если потоки executor-а будут забиты высокоприоритетными задачами, и наша не стартует, никто не заметит проблемы, очистка отработает на следующий запуск браузера или через час/день/месяц.

<br />

**Замечание**: Task-based орагнизация - более высокий уровень абстракции чем thread-based. 

**Замечание**: В домашнем задании про тайловую карту С++ вам нужно будет сделать именно task-based решение с фиксированным числом фоновых потоков.

<br />

##### atomic basics (since C++11)

Вспомним задачу параллельного суммирования массива из второй лекции на многопоточность.

Её реализация с ошибкой:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv += v[ix];
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

**Вопрос**: напомните, в чём ошибка?

<br />

Во второй лекции проблему решали через `mutex`.

**Вопрос**: как лучше всего решить задачу, чтобы синхронизаций было поменьше?

Сейчас будем писать тоже неидеальное решение - через `std::atomic`

Атомарность операции означает, что никакой из потоков не может отследить промежуточное состояние операции: либо состояние до изменений, либо состояние после изменений.

**Пример:**

`std::atomic<int>` имеет операцию `fetch_add`:

`var.fetch_add(int x)` имеет три условных стадии выполнения:

1. считать текущее значение из памяти в регистр
2. увеличить регистр на значение `x`
3. записать значение регистра в память

Никакой поток не может вклиниться в работу с `var`, пока выполняются шаги 1-2-3. Он либо работает с `var` до шага 1, либо после шага 3.


**Вопрос:** в примере ниже:
1. какое значение `counter` возможно после того как потоки выполняют свою работу?
2. если бы гарантии атомарности не было, какое значение мы могнли бы получить?

```c++
std::atomic<int> counter{0};

void thread_1_worker() { counter.fetch_add(1); }
void thread_2_worker() { counter.fetch_add(1); }

std::thread t1(thread_1_worker), t2(thread_2_worker);
t1.join(); t2.join();
```





Из документации по типу `atomic<T>`:

> Each instantiation and full specialization of the `std::atomic` template defines an atomic type. If one thread writes to an atomic object while another thread reads from it, the behavior is well-defined (see memory model for details on data races)

Какие типы `T` можно подставлять?

Обратимся опять к документации:

> The primary `std::atomic` template may be instantiated with any `TriviallyCopyable` type `T` satisfying both `CopyConstructible` and `CopyAssignable`. The program is ill-formed if any of following values is false:
* `std::is_trivially_copyable<T>::value`
* `std::is_copy_constructible<T>::value`
* `std::is_move_constructible<T>::value`
* `std::is_copy_assignable<T>::value`
* `std::is_move_assignable<T>::value`

Т.е. для любого `TriviallyCopyable` типа `T` объект типа `std::atomic<T>` потокобезопасен на чтение и запись.

```c++
struct Point
{
    float x;
    float y;
    float z;
};

std::atomic<Point> p;

void thread_1_worker() {
    p = Point{1.f, 2.f, 3.f};  // ok
}

void thread_2_worker() {
    Point x = p;  // ok
}
```

А вот эти примеры - ill-formed:

```c++
std::atomic<std::string> s;  // ill-formed
std::atomic<std::vector<int>> v;  // ill-formed
```

<br />

##### atomic vs mutex: performance

Для каких-то сложных типов (например, `Point`) `std::atomic` реализуется через `std::mutex` или аналогичным образом.

Если бы так было для всех типов, то особого смысла в `std::atomic` бы не было.

Смысл в том, что для простых типов (`int`, `bool`, `int64_t` ...) многие CPU поддерживают более дешёвые способы синхронизации. Набор типов и степень их дешевизны зависят от архитектуры CPU.

Напишем реализацию паралелльной суммы через `std::atomic<int>`:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    std::atomic<int> rv{0};

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv.fetch_add(v[ix]);
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

И через `std::mutex`:

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    int rv = 0;
    std::mutex m;

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, &m, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
            {
                std::lock_guard guard(m);
                rv += v[ix];                
            }
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

И сравним производительность на 6 потоках (тестовая машинка: 6 физических ядер Intel Core i5-8400):

```sh
g++ parallel_sum_atomic.cpp -lpthread -O3 -std=c++17 -o sum_atomic.exe && ./sum_atomic.exe
g++ parallel_sum_mutex.cpp  -lpthread -O3 -std=c++17 -o sum_mutex.exe  && ./sum_mutex.exe
```

вывод:

```sh
parallel sum atomic:
  size          = 60000000
  threads_count = 6
  result        = 60000000
  time, sec     = 1.60537
 
parallel sum mutex:
  size          = 60000000
  threads_count = 6
  result        = 60000000
  time, sec     = 6.33526
```

В зависимости от примитива синхронизации получилось добиться 4-х кратного ускорения алгоритма, что существенно.
Такой эффект достигается за счёт того, что в алгоритме слишком агрессивно используется синхронизация данных.

<br />

Чтобы оценить что происходит в бинарном коде закинем файл на godbolt.org (gcc 9.2 -O3 -std=c++17):

```c++
#include <atomic>
#include <mutex>

int mutexed_counter = 0;
std::mutex m;

std::atomic<int> atomic_counter{0};

void add_mutexed(int value)
{
    std::lock_guard guard{m};
    mutexed_counter += value;
}

void add_atomic(int value)
{
    atomic_counter += value;
}
```

**Вопрос**: Перед тем как посмотреть на ответ, подскажите, что происходит внутри `lock_guard` и `mutex` в этом коде?

```asm
add_mutexed(int):
        push    rbp
        mov     ebp, OFFSET FLAT:_ZL28__gthrw___pthread_key_createPjPFvPvE
        push    rbx
        mov     ebx, edi
        sub     rsp, 8
        test    rbp, rbp
        je      .L2
        mov     edi, OFFSET FLAT:m
        call    __gthrw_pthread_mutex_lock(pthread_mutex_t*)
        test    eax, eax
        jne     .L12
.L2:
        add     DWORD PTR mutexed_counter[rip], ebx
        test    rbp, rbp
        je      .L1
        add     rsp, 8
        mov     edi, OFFSET FLAT:m
        pop     rbx
        pop     rbp
        jmp     __gthrw_pthread_mutex_unlock(pthread_mutex_t*)
.L1:
        add     rsp, 8
        pop     rbx
        pop     rbp
        ret
.L12:
        mov     edi, eax
        call    std::__throw_system_error(int)
        
add_atomic(int):
        lock add        DWORD PTR atomic_counter[rip], edi
        ret
```

Внутри `add_mutexed` можно наблюдать уход в ядро ОС через `pthread`.

А весь `add_atomic` - одна инструкция `lock add` - особая атомарная инструкция сложения.

<br />

##### atomics: hardware

Сначала рассмотрим hardware-нюансы обзорно.

Вспомним многоуровневую организацию кешей памяти:

<img src="cpu_caches_ram.png" width=50% height=50% />

При работе многопоточного приложения одна и та же ячейка памяти может оказаться одновременно в разных кешах L1 разных CPU.

Если при этом происходит только чтение из памяти, то проблем не возникает. Возникают проблемы, когда нужна запись.

Записанное значение должно "просочиться" по всем уровням иерархии кешей вплоть до RAM... и обновиться в кешах соседних CPU.

Процесс "просачивания" не быстрый, другие CPU могут читать устаревшее значение или попытаться записать своё.

Особые atomic-инструкции решают эту проблему, они гарантируют, что записанное одним CPU значение "просочится" по всей иерархии кешей и будет корректно прочитано другими CPU.

Поэтому атомарные инструкции (как правило - зависит от железа) медленнее аналогичных неатомарных инструкций:
  * `+=` для `std::atomic<int>` будет медленнее чем для `int`
  * также работа с `std::atomic<int>` отключает некоторые оптимизации компилятора (подробности в разделе про instruction reordering && memory model)

Это упрощённое описание. Пока что его будет достаточно для дальнейшей работы. Большие нюансы о гарантиях и оптимизации atomic - раздел про memory model, о нём позже.

<br />

##### instructions reordering

* https://preshing.com/20120625/memory-ordering-at-compile-time/
* https://preshing.com/20120710/memory-barriers-are-like-source-control-operations/

Переупорядочивание инструкций - изменение порядка выполняемых команд "без видимых эффектов" - одна из оптимизаций программ, будучи полностью прозрачной для однопоточного кода, начинает играть роль для некорректного многопоточного кода (для корректного всё хорошо).

Важно, что _ассемблерные_ инструкции не могут быть переупорядочены, если между ними есть зависимость по данным.

Переупорядочивание инструкций можно разделить на 2 типа:
* в compile time
* в runtime

**compile-time**

Во-первых, компилятор может решить, что лучше команды выполнять не в той последовательности, в которой их описал программист.

Рассмотрим такой код:

```c++

double some_value = 0.;
bool value_is_set = false;


void run_setup()
{
    some_value = 3.14;    // между some_value и value_is_set нет зависимости
    value_is_set = true;  // по данным, компилятор вправе поменять присванивания местами
}

void process()
{
    if (value_is_set)
        assert(some_value == 3.14);
}
```

В однопоточном приложении здесь всё будет хорошо, а в многопоточном и так плохо т.к. нет нужных синхронизаций, но переупорядочивание инструкций может увеличить вероятность ошибки.


Другой пример с переупорядочиванием инструкций, когда зависимость по данным есть в плюсовом коде, но её нет на увроне инструкций ассемблера:

```c++
int A, B;

void foo()
{
    A = B + 1;
    B = 0;
}
```

Здесь компилятор так же вправе сгенерировать запись в ячейку памяти B раньше чем в A:

либо так:

```asm
mov     eax, DWORD PTR _B  (redo this at home...)
add     eax, 1
mov     DWORD PTR _A, eax
mov     DWORD PTR _B, 0
```

либо так:

```asm
mov     eax, DWORD PTR B
mov     DWORD PTR B, 0
add     eax, 1
mov     DWORD PTR A, eax
```

Опять же, для однопоточного кода такие трюки должны быть прозрачны. Они начинают играть свою роль только для многопоточного кода.

**Замечание**: есть способ запретить копилятору переупорядочивать некоторые операции, добавив барьер, [подробнее - читайте по ссылке выше](https://preshing.com/20120625/memory-ordering-at-compile-time/)

**runtime**

Во-вторых, даже если компилятор сохранил последовательность команд программиста, CPU может иметь своё мнение, и решить, что быстрее будет их выполнить в ином порядке.

Рассмотрим пример такого устройства:

![](cpu_mem_organization.jpg)

В этой схеме у каждого ядра "своя личная область работы" - кеш L1, а "общая область" - кеш L2 + RAM.

Рассмотрим случай, когда в какой-то программе поток 1 выполняет код на ядре 1, а поток 2 выполняет код на ядре 2:

```c++
int x = 0, y = 0, z = 0;
int a = 0, b = 0, c = 0;

void thread_1_worker_on_cpu_1() {
    x = 1; y = 2; z = 3;
    
    if (b == 5)
        assert(a == 4);  // fail
}

void thread_2_worker_on_cpu_2() {
    a = 4; b = 5; c = 6;

    if (y == 2)
        assert(x == 1);  // fail
}
```

В таком коде нет гарантий на порядок записи / чтения между "личной областью работы" и "общей областью работы".

* Порядок, кто из `x`, `y`, `z` раньше попадёт в L2 + RAM не определён, так же как и время, через которое они туда попадут.
  * вполне может оказаться так, что z улетит в RAM первым, а x и y ещё долго провисят недоставленными по адресу в L1
* Аналогично, не определён порядок попадания `a`, `b`, `c` в "общую область работы"
* Хуже того, не определён порядок попадания `x`, `y`, `z` в "личную область работы" потока 2, т.е. при чтении тоже нет гарантий, что поток 2, увидев `y == 2`, получит `x == 1`
* Ещё хуже, если запустить поток 3 на cpu 3, он может увидеть совсем другую последовательность значений `x`, `y`, `z`, чем поток 2

При таких условиях многопоточных код писать невозможно. Чтобы справляться с этими проблемами есть особые барьеры памяти (их много и разных видов), о них поговорим чуть позже.

Если коротко, без использования барьеров памяти:
* значения на запись уходят в "общую область" сколь угодно долго и в произвольном порядке
* значения на чтение приходят в "личную область" сколько угодно долго и в произвольном порядке

<br />

##### double checked locking

https://preshing.com/20130930/double-checked-locking-is-fixed-in-cpp11/

http://www.cs.umd.edu/~pugh/java/memoryModel/DoubleCheckedLocking.html

https://www.aristeia.com/Papers/DDJ_Jul_Aug_2004_revised.pdf

DCLP - double checked locking pattern - популярный шаблон организации потокобезопасного доступа к данным/объектам с ленивой инициализацией. Пример: синглтоны с ленивой инициализацией.

**Вопрос-напоминалка**: что такое синглтон?

Однопоточный вариант организации ленивого синглтона:
    
```c++
class Singleton
{
    static Singleton* object;

public:
    static Singleton* instance()
    {
        if (!object)
            object = new Singleton;
        return object;
    }
};
```

**Вопрос**: Почему эта реализация не работает в многопоточной среде? Приведите пример последовательности выполнения потоков, при котором возникает ошибка.

<br />

Многопоточный вариант реализации ленивого синглтона:
    
```c++
class Singleton
{
    static Singleton* object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        std::lock_guard guard(mtx);
        
        if (!object)
            object = new Singleton;
        return object;
    }
};
```

**Вопрос**: корректна ли эта реализация? Какие у неё проблемы?

<details>
<summary>ответ</summary>
<p>

Корректна, но очень дорого: когда синглтон инициализирован каждый из потоков спотыкается об `mutex` при доступе к синглтону. `mutex`-ы уходят в kernel space, хочется что-нибудь подешевле.

</p>
</details>

<br />

Вариант double checked locking pattern:

```c++
class Singleton
{
    static Singleton* object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        if (!object)  // check 1
        {
            std::lock_guard guard(mtx);

            if (!object)  // check 2
                object = new Singleton;
        }
        return object;
    }
};
```

**Вопросы**: (чтобы тщательно разобрать код)
* зачем нужна вторая проверка, будет ли код корректен без неё?
* корректен ли код?

<br />

Со временем в этом шаблоне нашли хитрые race condition, что отчасти и подтолкнуло сообщество к формализации memory model в языках Java и С++.
* до Java 2004 DCLP не работал [(подробности)](http://www.cs.umd.edu/~pugh/java/memoryModel/DoubleCheckedLocking.html)
* до С++11 не было портируемого способа реализовать DCLP (стандарт не давал необходимых для этого инструментов и гарантий) 

Проблема в этой строчке:
    
```c++
object = new Singleton;
```

Программист мог бы подумать, что этот код выполняется в такой последовательности:
* выделить память под `Singleton`
* позвать конструктор `Singleton`
* записать указатель в `object`

Здесь в игру вступает instructions reordering.

Например, если компилятор может доказать, что конструктор `Singleton` не бросает исключений, он вправе сгенерировать такой код:

```c++
class Singleton
{
    static Singleton* object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        if (!object)  // check 1
        {
            std::lock_guard guard(mtx);

            if (!object)  // check 2
            {
                object = operator new(sizeof(Singleton));
                new (object) Singleton;
            }
        }
        return object;
    }
};
```

**Вопрос:** в чём здесь проблема? как её можно поймать?

Как это пытались починить различными способами до стандартизации модели памяти, и почему они не работают - [в статье от Мейерса и Александреску](https://www.aristeia.com/Papers/DDJ_Jul_Aug_2004_revised.pdf)

<br />

Корректная реализация double checked locking pattern будет выглядеть так:

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load();

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load();
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp);
            }
        }
        return tmp;
    }
};
```

В этом примере методы `load`/`store` у атомарных типов гарантируют корректный порядок операций между потоками.

Как именно получаются гарантии и как сделать реализацию быстрее - рассмотрим позже, когда поговорим про memory model.

<br />

Если мы хотим ленивую инициализацию синглтона, то 11-ый стандарт даёт два варианта, которые скрывают проблемы DCLP от программиста:

**Вариант 1**: специальная конструкция для ленивых многопоточных синглтонов - `static`-переменные внутри функции:

```c++
class Singleton
{
public:
    static Singleton& instance()
    {
        static Singleton object;
        return object;
    }
};
```

`static`-переменные внутри функций/методов (не путать с глобальными `static`-переменными!):
* инициализируются лениво при первом обращении
* компилятор с поддержкой С++11 обязан сгенерировать потокобезопасную инициализацию переменной. Компилятор не знает, будет этот код использован в многопоточном варианте или в однопоточном, поэтому он всегда генерирует безопасный многопоточный вариант
* компилятор не обязан использовать DCLP, он может выбрать любой способ, который на его взгляд, работает лучше для данного случая, поэтому:

```c++
#include <string>

float get_circle_area(const float radius)
{
    // скорее всего, здесь не будет никаких синхронизаций,
    // но и static в этом месте - лишнее, лучше удалить
    static const float pi = 3.14f;
    return pi * radius * radius;
}

std::string add_12345(const std::string& x)
{
    // скорее всего, здесь будет синхронизация
    static const std::string s = "12345";
    return x + s;
}
```

Закинуть на godbolt.org этот пример, показать.

**Вопрос на понимание**: в чём разница этих двух решений?

```c++
std::string add_12345_1(const std::string& x)
{
    static const std::string s = "12345";
    return x + s;
}
```

и

```c++
static const std::string s = "12345";

std::string add_12345_2(const std::string& x)
{
    return x + s;
}
```

<br />

**Вариант 2**: `std::call_once` (since C++11)

https://en.cppreference.com/w/cpp/thread/call_once



```c++
template< class Callable, class... Args >
void call_once( std::once_flag& flag, Callable&& f, Args&&... args );
```

> Executes the Callable object `f` exactly once, even if called concurrently, from several threads.

`std::call_once` позволяет гарантировать, что какая-то работа будет выполнена только один раз в многопоточной среде.

Более подробно про использование:

```c++
// какая-то работа, единоразовое выполнение которой нужно гарантировать
void some_job(int param) {  /*...*/ }

// где-то завели флажок, была ли выполнена работа
// (специальный тип std::once_flag)
std::once_flag my_flag;

// вызываем std::call_once в любом потоке:
std::vector<std::thread> threads;
for (int i = 0; i != 5; ++i)
    threads.emplace_back([&once_flag, i](){
        /* .. */
        
        std::call_once(once_flag, some_job, i);
        
        /* ... */        
    });

for (auto& t : threads)
    t.join();
```

В этом примере гарантировано, что `some_job` *отработает* только единожды (с каким `param` - неизвестно, с которым первым получится). Когда `some_job` *отработает*, флаг `my_flag` будет взведён, и больше `some_job` не будет вызываться.

<br />

**Вопрос**: какие в С++ есть (адекватные) стратегии обработки ошибок?

<details>
<summary>ответ</summary>

код возврата и исключения

</details>

<br />

`std::call_once` немножко хитрее, он умеет понимать, произошла ли ошибка в `some_job`. Если в `some_job` произошла ошибка, то он не считает, что работа выполнена.

Ошибку `std::call_once` определяет по тому, вылетело ли из `some_job` исключение или выполнение функции завершилось стандартным способом.
* Если выполнение завершается стандартным способом - флаг взводится, считается, что работа выполнена
* Если вылетает исключение:
  * флаг НЕ взводится
  * исключение пролетает сквозь `std::call_once` наружу тому, кто `std::call_once` позвал 
  * следующий вызов `std::call_once` снова попытается выполнить работу

<br />

Разберём пример:

```c++
#include <atomic>
#include <exception>
#include <iostream>
#include <mutex>
#include <sstream>
#include <string>
#include <thread>
#include <vector>

std::atomic_int attempts_count{0};

std::string s;

void print_thread_msg(const std::string& message)
{
    std::stringstream ss;
    ss << "thread " << std::this_thread::get_id() << ' ' << message << '\n';
    std::cout << ss.str();
}

void construct_object()
{
    const int before_add = attempts_count.fetch_add(1);
    if (before_add < 5)
    {
        print_thread_msg("gets an attempt ix = " + std::to_string(before_add) + " and throws an exception");
        throw std::runtime_error("try again");
    }

    print_thread_msg("gets an attempt ix = " + std::to_string(before_add) + " and constructs the value");

    s = "hello c++";
}

int main()
{
    std::once_flag once_init_flag;

    std::vector<std::thread> threads;
    for (int i = 0; i != 10; ++i)
        threads.emplace_back([&](){
            try
            {
                std::call_once(once_init_flag, construct_object);
            }
            catch (const std::exception&) {}
        });

    for (auto& thr: threads)
        thr.join();

    std::cout << "value is: " << s << std::endl;

    return 0;
}
```

**Вопросы**:
* сколько потоков создаётся?
* сколько войдёт в `construct_object`?
* какой ожидается вывод?
* зачем отдельно вынесен такой страшный `print_thread_msg`?

Скомпилируем пример clang-ом с clang-овской стандартной библиотекой libc++ и запустим:
    
```sh
clang++-8 -O2 call_once.cpp -lpthread -stdlib=libc++
```

Вывод:

```sh
thread 139683658577664 gets an attempt ix = 0 and throws an exception
thread 139683641792256 gets an attempt ix = 1 and throws an exception
thread 139683650184960 gets an attempt ix = 2 and throws an exception
thread 139683535251200 gets an attempt ix = 3 and throws an exception
thread 139683510073088 gets an attempt ix = 4 and throws an exception
thread 139683526858496 gets an attempt ix = 5 and constructs the value
value is: hello c++
```

Обратите внимание, что после того как функция `construct_object`, переданная в `std::call_once`, завершила свою работу без пробрасывания исключений, больше никто `construct_object` не вызывает - работает гарантия `std::call_once`.

Теперь скомпилируем тот же самый код gcc со стандартной билиотекой libstdc++ и запустим:

```sh
g++ -O2 call_once.cpp -lpthread
```

И он... зависнет:

```sh
thread 139670308796160 gets an attempt ix = 0 and throws an exception
# зависание тут
```

**Вопрос**: попробуйте догадаться, где ошибка?

<details>
<summary>ответ</summary>
    
В нашем коде ошибки нет, это [баг библиотеки libstdc++](https://gcc.gnu.org/bugzilla/show_bug.cgi?id=66146), известный ещё с 2015 года. В комбинации libstdc++ + posix threads не поддержана работа с исключениями в `std::call_once`. Можно для интереса зайти [на документацию `std::call_once`](https://en.cppreference.com/w/cpp/thread/call_once) и попробовать прогнать пример из документации.

Мораль - даже если вы написали идеальный код по стандарту, не забудьте его протестировать перед тем как <s>показать студентам</s> выпустить в production.

Тесты - хоть какая-то уверенность.
    
</details>

<br />

**Резюме** по блоку double checked locking pattern:
* instructions reordering может принести неожиданные сюрпризы в многопоточных программах
* гарантии на последовательность операций дают `std::atomic`-операции со стандарта С++11
* если нужно лениво инициализировать глобальную переменную / константу, предпочтительнее использовать `static`-переменную внутри функции / метода, хороший компилятор выберет наиболее предпочтительный способ синхронизации для вашего случая.
* если нужно гарантировать разовое выполнение работы в многопоточной среде - используйте связку `std::call_once + std::once_flag`

Сравнение производительности `static` / `call_once` и других подходов:
    
http://www.modernescpp.com/index.php/thread-safe-initialization-of-a-singleton
    
Спойлер: `static` всех победил

<br />

##### spinlock && гибриды

**Вопрос**: как устроен внутри `std::mutex`, что происходит при вызовах методов `lock`/`unlock`?

<br />

В некоторых случаях использование `std::mutex` может оказаться слишком дорогим.

Рассмотрим случай, когда защищаемый участок кода выполняется очень быстро:

```c++
std::vector<int> v;
std::mutex mtx;

void add_item(int item)
{
    std::lock_guard guard(mtx);
    v.push_back(item);
}
```

Зачастую `v.push_back` отрабатывает очень быстро, всего несколько инструкций: увеличить размер на 1 элемент и записать целое в область памяти. Время его работы значительно меньше, чем двойной уход в kernel space для `std::mutex` при вызовах `lock`/`unlock`. В такой функции всё время работы уходит на синхронизацию.

Хочется подешевле.

**Вопрос**: какой вариант после знакомства с сегодняшней лекцией вы могли бы предложить для таких блокировок?

<br />

`spinlock` - вариант решения проблемы на атомарных операциях.

Простейшая его реализация:
* атомарный флаг занятости
* в `lock` подождать, пока флаг не станет "свободным", выставить "занято"
* в `unlock` выставить "свободно"

Вариант реализации (если понимать memory model, можно реализовать быстрее):

https://en.cppreference.com/w/cpp/atomic/atomic_flag

```c++
class spin_lock
{
    std::atomic_flag busy = ATOMIC_FLAG_INIT;
    
public:
    void lock()
    {
        // test_and_set - устанавливает флаг в true и
        // возвращает предыдущее значение
        //
        // (если вернуло false, значит, это мы изменили
        //  флаг false -> true)
        
        while (busy.test_and_set())  // acquire lock
             ; // spin
    }
    
    void unlock()
    {
        busy.clear();  // release lock
    }
};

// Вопрос: почему это работает? как два потока будут бороться за lock?
```

И потом можно сделать так:

```c++
std::vector<int> v;
SpinLock spin_lock;

void add_item(int item)
{
    std::lock_guard guard(spin_lock);
    v.push_back(item);
}
```

и `add_item` начнёт работать значительно быстрее.

<br />

**Вопрос**: Если бы spinlock был так хорош, то можно было бы просто все `std::mutex` позаменять на spinlock и не мучиться. В чём проблема spinlock, чем он хуже `mutex`?

<details>
<summary>ответ</summary>
<p>

* `std::mutex` усыпляет висящий на его `lock`-е поток средствами ОС. Пока поток висит на `std::mutex`, ОС закинет на физическое ядро другие потоки, выполняющие полезную работу
* `spinlock` оккупирует ядро в `lock` в цикле `while`, никому не отдавая свой фрейм выполнения. Если `spinlock` висит долго, он впустую крутит физическое ядро вместо того чтобы отдать ресурсы на полезную работу.

</p>    
</details>

<br />

**Поэтому для простых решений выработано правило:**
* если блокируемый участок кода выполняется быстро - `spinlock`
* если блокируемый участок кода выполняется медленно - `std::mutex`

<br />

Есть варианты частичного исправления проблемы `spinlock`-а - `spinlock` с засыпанием. Идея реализации метода `lock`:
* покрутиться недолго на атомике в ожидании значения "свободно"
* усыпить поток на время `x` (через `std::this_thread::yield`, например)
* покрутиться недолго на атомике в ожидании значения "свободно"
* усыпить поток на время `2 * x`
* покрутиться недолго на атомике в ожидании значения "свободно"
* усыпить поток на время `4 * x`
* покрутиться недолго на атомике в ожидании значения "свободно"
* усыпить поток на время `8 * x`
...

т.е. после некоторой оккупации физического ядра и кручения его на спинлоке всё-таки принять решение отдать ядро кому-то другому, а самому уснуть. Время засыпания увеличивать в геометрической прогрессии до некоторой верхней границы.

Тоже допустимое решение, но `std::mutex` и здесь может начать выигрывать на длительных ожиданиях:
* во время `std::this_thread::yield` поток не знает, когда проснуться, он будет спать запрошенное время, а событие разблокировки, например, произойдёт в середине сна.
* к тому же остаются расходы на оккупацию физического ядра, но они меньше
* в случае с `std::mutex` ОС знает, когда нужно разбудить ожидающий поток

<br />

В случае, когда объём вычислений под блокировкой предсказать сложно (или не хочется думать), иногда используют гибридный вариант блокировки:
* сначала покрутиться несколько раз на `atomic_flag`
* если через atomic разрулить не получилось, уйти через `std::mutex` в kernel space, и пусть ОС сама решает когда нас разбудить

Так, например, сделан [Webkit WTF::Lock](https://webkit.org/blog/6161/locking-in-webkit/) (статья объёмная, но полезная, почитайте)

<br />

##### intro to lock free programming

https://preshing.com/20120612/an-introduction-to-lock-free-programming
    
Jeff Pershing даёт следующее определение lock-free:

> lock free programming = multithreaded app + shared memory + threads do not lock each other

Под "threads do not lock each other" понимается, что при любом причудливом планировании распределения фреймов cpu по потокам работа будет продвигаться. Если даже ОС полностью усыпила какой-то один поток, остальные продолжат выполнять работу по алгоритму.

**Вопрос**: если принять такое определение, почему любой алгоритм с использованием mutex не является lock free?

**Замечание**: к такому определению следует относиться с некоторой долей скепсиса и понимания его неидеальности. Предположим, у нас есть lock-free очередь задач с одним потоком, создающим заадачи и несколькими потоками, их выполняющими (например, ваша домашняя работа с тайловой картой). Почему такая очередь не совсем lock-free?

<br />

**Пример:** пример без мьютексов с атомарным `х` равным 0 на входе, который НЕ lock free:
    
```c++
while (X == 0)
    X = 1 - X;
```

**Вопрос:** каким образом два апотока могут здесь друг друга залочить

**Следствие:** отсутствие `mutex`-ов и `spin_lock`-ов ещё не делает алгоритм lock free

<br />

Другое определение lock free:
    
> as long as the program is able to keep calling those lock-free operations, the number of completed calls keeps increasing, no matter what

<br />

Если по какой-то причине после сегодняшней лекции (её примеров и упрощений) вам покажется, что lock-free - простая тема, [здесь](https://preshing.com/20131125/acquire-and-release-fences-dont-work-the-way-youd-expect/) пример как Jeff Pershing находит ошибку в презентации про атомики и барьеры от Герба Саттера (генсека <s>КПСС</s> КСС++, второго человека в мире С++).

<br />

##### hardware memory model (simplified)

https://preshing.com/20120930/weak-vs-strong-memory-models/

memory ordering - правила упорядочивания операций с памятью можно разделить на:

* правила языка для абстрактной машины
    * и вы программу пишите с ними и только под них
 
* hardware-специфичные правила
    * применяются в зависимости от железа, на котором выполняется программа - под них программу писать не надо, но желательно понимать, чтобы понимать, каких ошибок можно ожидать от железа, когда в программе наошибались в правилах языка

Поговорим про hardware memory ordering

Варианты:
* really weak
* weak with data dependency
* usually strong
* sequentially consistent

![](weak-strong-table-4.png)

**really weak**

(Без специальных барьеров) нет никаких гарантий:

Вспомним пример:

```c++
int x = 0, y = 0, z = 0;
int a = 0, b = 0, c = 0;

void thread_1_worker_on_cpu_1() {
    x = 1; y = 2; z = 3;

    if (b == 5)
        assert(a == 4);  // fail
}

void thread_2_worker_on_cpu_2() {
    a = 4; b = 5; c = 6;

    if (y == 2)
        assert(x == 1);  // fail
}
```

Нет никаких гарантий на порядок/время записи данных в "общую память (L2 + RAM)", ровно как и на порядок/время их просачивания до "частной памяти ЦПУ (L1)".


**weak with data dependency**

Появляется гарантия (из статьи Джеффа):

> It means that if you write A->B in C/C++, you are always guaranteed to load a value of B which is at least as new as the value of A

на примере:

```c++
int x = 0, y = 0;
int a = 0, b = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;
    y = x;  // <---- в одну ячейку памяти пишем содержимое другой
    
    a = 1;
    b = 1;  // больше x, y, a, b никто не трогает
}

void thread_2_worker_on_cpu_2() {
    if (y == 1)
        assert(x == 1);  // ok

    if (b == 1)
        assert(a == 1);  // fail
}
```

**Замечание:** У меня есть сомнения про это утверждение Джеффа, т.к. вроде бы ничто не мешает компилятору соптимизировать до `y = 1` ещё до того как hardware memory model вступает в игру.

**strong model**

> A strong hardware memory model is one in which every machine instruction comes implicitly with acquire and release semantics. As a result, when one CPU core performs a sequence of writes, every other CPU core sees those values change in the same order that they were written.

**Замечание**: Несмотря на то, что к strong model бодро отнесён x86/64, не все операции в x86/64 поддерживают strong memory model, читайте исходную статью и комментарии.

**Замечание**: ниже написан код плюсовый, но сделаем вид, будто компилятор ничего переупорядочивать не будет, если будет - всё неверно. Мы как будто бы пишем на ассемблере в синтаксисе плюсов.

```c++
int x = 0, y = 0, z = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;
    y = 2;
    z = 3;
}

void thread_2_worker_on_cpu_2() {
    if (z == 3) {
        assert(y == 2);  // ok
        assert(x == 1);  // ok
    }
}
```

В strong модели разрешено переупорядочивание операций store-load (запись-чтение) (но запрещены load-store, store-store && load-load)

```c++
int x = 0, y = 0;
int a = 0, b = 0;

void thread_1_worker_on_cpu_1() {
    x = 1;  // store x
    a = y;  // load y
            // store a
}

void thread_2_worker_on_cpu_2() {
    y = 1;  // store y
    b = x;  // load x
            // store b
}
```

В результате такого "ассемблера" может получиться `a = 0 && b = 0`.

**Вопрос**: как?

<details>
<summary>ответ</summary>

* cpu1: register1 = read y ( == 0)
* cpu2: register2 = read x ( == 0)
* cpu1: x = 1
* cpu2: y = 1
* cpu1: a = register1 ( == 0)
* cpu2: b = register2 ( == 0)
    
    
</details>


**sequential consistent**

Любое переупорядочивание запрещено, из примера выше `a = 0 && b = 0` не получить.

<br />

##### C++ memory model (simplified, since C++11)

https://en.cppreference.com/w/cpp/atomic/memory_order

https://preshing.com/20120913/acquire-and-release-semantics

Теперь поговорим про memory ordering на уровне языка С++ - те, с которыми вам необходимо работать при написании программ.

В стандартной библиотеке С++ определён enum для различного рода ограничений на переупорядочивание операций с памятью.

```c++
typedef enum memory_order {
    memory_order_relaxed,
    memory_order_consume,
    memory_order_acquire,
    memory_order_release,
    memory_order_acq_rel,
    memory_order_seq_cst
} memory_order;
```

Из них рассмотрим:

```c++
memory_order_relaxed
memory_order_acquire
memory_order_release
memory_order_seq_cst
```

<br />

`memory_order_release` означает, что все операции с памятью до `release`-барьера, не перейдут через барьер "вперёд".

```c++
x = 1;  // store x
y = a;  // load a
        // store y

/*release barrier*/

...  // операции, написанные выше с x, y и a
     // не могут по времени произойти после release-барьера
```

`memory_order_acquire` означает, что все операции с памятью после `acquire`-барьера, не перейдёт через барьер "назад".

```c++
...  // операции, написанные ниже с x, y и a
     // не могут по времени произойти раньше acquire-барьера

/*acquire barrier*/

x = 1;  // store x
y = a;  // load a
        // store y
```

**Пример:** разобрать очень подробно про гарантии порядка вычислений. Вопрос: почему assert корректен?

```c++
std::atomic<bool> is_published{false};
int data = 0;

void thread_1_worker()
{
    data = 1;
    is_published.store(true, std::memory_order_release);
}

void thread_2_worker()
{
    if (is_published.load(std::memory_order_acquire))
        assert(data == 1);  // ok
}
```

Имена `release` и `acquire` подобраны по смыслу операции:
* `release` - опубликовать данные
* `acquire` - принять опубликованные данные

<br />

`memory_order_seq_cst` - операции не могут перескочить через барьер ни в какую сторону (аналог sequential consistent hardware memory model) - самый строгий и самый медленный вариант

`memory_order_seq_cst`:
* дефолтный способ упорядочивания в `atomic`-операциях (чтобы программисты меньше ошибались)
* аналог `volatile` в Java 5+



```c++
x = 1;  // store x
y = a;  // load a
        // store y

/*seq_cst barrier*/  // никакая из операций не может перескочить через барьер

z = 2;  // store x
b = c;  // load c
        // store b
```

<br />

`memory_order_relaxed`:
* возможны любые переупорядочивания операций с памятью (нет барьеров)
* гарантируется только атомарная модификация защищённого объекта

**Пример:**

```c++
std::atomic<int> atomic_var{0};

// нет гарантий на порядок операций с памятью,
// только атомарность работы с atomic_var
void thread_worker() {
    x = 1;
    y = a;
    atomic_var.store(2, std::memory_order_relaxed); 
}
```

<br />

Разница в гарантиях порядка необходима из-за различной стоимости барьеров в CPU: чем строже гарантии, тем медленнее могут выполняться команды.

* `relaxed` - самый быстрый (нет барьеров, только атомарность операции)
* `acquire/release` - медленнее или аналогично `relaxed` (барьеры в одну сторону + атомарность операции)
* `seq_cst` - медленнее или аналогично `acquire/release` (барьеры в обе стороны + атомарность операции)

<br />

##### оптимизированный DCLP

Напомню DCLP, на котором мы закончили его рассмотрение:

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load();

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load();
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp);
            }
        }
        return tmp;
    }
};
```

**Вопрос:** как, зная про memory ordering, сделать более быструю реализацию?

**Ответ:**

```c++
class Singleton
{
    static std::atomic<Singleton*> object;
    static std::mutex mtx;

public:
    static Singleton* instance()
    {
        auto* tmp = object.load(std::memory_order_acquire);  // <--- mem order on read data

        if (!tmp)  // check 1
        {
            std::lock_guard<std::mutex> lock(mtx);

            tmp = object.load(std::memory_order_acquire);  // <--- mem order on read data
            if (!tmp)  // check 2
            {
                tmp = new Singleton;
                object.store(tmp, std::memory_order_release);  // <--- mem order on publish data
            }
        }
        return tmp;
    }
};
```

<br />

##### оптимизированная сумма элементов в массиве

Напомню как мы считали сумму элементов в массиве:
    
```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    std::atomic<int> rv{0};

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv.fetch_add(v[ix]);
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

**Вопрос:** как, зная про memory ordering, сделать более быструю реализацию?

**Ответ:**

```c++
int parallel_sum(const std::vector<int>& v, const unsigned threads_count)
{
    const unsigned len = v.size() / threads_count;
    assert(len * threads_count == v.size());

    std::atomic<int> rv{0};

    std::vector<std::thread> threads;
    for (unsigned i = 0; i < threads_count; ++i)
        threads.emplace_back([&v, &rv, i, len](){
            for (unsigned ix = len * i, final_ix = len * (i + 1); ix < final_ix; ++ix)
                 rv.fetch_add(v[ix], std::memory_order_relaxed);  // <---- no barriers, just atomic update
        });

    for (auto& t: threads)
        t.join();

    return rv;
}
```

<br />

##### lock free stack

На лекции мы реализуем lock free стэк с ошибками на базе списка и разберём ошибки.

Как делать правильный - начните разбирать [с этой статьи](https://habr.com/ru/post/216013/) и далее по ссыкам.

<br />

**Шаг 1:** нарисовать стэк на доске и обсудить как сделать lock-free операции `push`/`pop`

<br />

**Шаг 2:** накидаем примерную реализацию (объяснить каждую строчку)

```c++
template<typename T>
class Stack {
public:
    struct Element
    {
        T data;
        std::atomic<Element*> next;
    };
    
private:
    std::atomic<Element*> top;

public:
    bool push(value_type& val)
    {
        auto* t = top.load();
        while (true)
        {
            val.next.store(t);
            if (top.compare_exchange_strong(t, &val))       
               return true;
        }
    }
    
    Element* pop()
    {
       while (true)
       {
          auto* t = top.load();
          if (!t)
             return nullptr ;  // stack is empty

          auto* next = t->next.load();
          if (top.compare_exchange_strong(t, next))
          {
              return /* somehow free memory for element t and return value */;
          }
       }
    }
};
```

**Замечание:** если корректно заменить seq_cst операции на более слабые аналоги с relaxed && acquire/release порядком, то можно получить более быстрый код. Сейчас этим заниматься не будем

**Вопрос:** где в коде ошибка?

<details>
<summary>ответ</summary>
<p>

тут:

```c++
auto* next = t->next.load();
```

как её поймать?

Чтобы починить, нужно использовать специальные структуры данных, о них - по ссылке на правильную реализацию

</p>
</details>

**Замечание:** ещё в коде есть ABA-ошибка. (Объяснить её)

Как чинить ABA-ошибки, [читайте в работе Страуструпа](http://www.stroustrup.com/isorc2010.pdf).

<br />

**Резюме**:
* вы не хотите писать lock-free алгоритмы в production

<br />

**Неохваченные темы по параллельному программированию:**
* процессы
* fibers
* futexes
* semaphore
* livelock
* thread starving && fair algorithms
* openmp введение
* mpi введение